In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# Define headers to mimic a real browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

# A class to represent a Webpage
class Website:

    def __init__(self, url):
        """
        This class creates a website object from the given URL using BeautifulSoup or Selenium depending on the content type
        """
        self.url = url
        # Check if the page is dynamic or static
        if self.is_dynamic_page(url):
            self.text = self.get_dynamic_content(url)
        else:
            self.text = self.get_static_content(url)
        self.title = self.get_title()

    def get_title(self):
        """
        Fetches the title of the page
        """
        return self.soup.title.string if self.soup.title else "No title found"

    def get_static_content(self, url):
        """
        Fetches static content from the page using requests and BeautifulSoup
        """
        response = requests.get(url, headers=headers)
        self.soup = BeautifulSoup(response.content, 'html.parser')
        # Remove irrelevant tags like script, style, img, and input
        for irrelevant in self.soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        return self.soup.body.get_text(separator="\n", strip=True)

    def get_dynamic_content(self, url):
        """
        Fetches dynamic content from the page using Selenium
        """
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        driver.get(url)
        time.sleep(3)  # Wait for JavaScript to load the content
        # Get the page source after the JavaScript has been executed
        self.soup = BeautifulSoup(driver.page_source, 'html.parser')
        driver.quit()  # Close the browser
        # Remove irrelevant tags like script, style, img, and input
        for irrelevant in self.soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        return self.soup.body.get_text(separator="\n", strip=True)

    def is_dynamic_page(self, url):
        """
        A helper function to determine if the page is dynamic or static (based on URL pattern or other logic)
        """
        # For simplicity, we're assuming pages with 'javascript' in the URL are dynamic (this can be improved)
        return "javascript" in url or "dynamic" in url

In [11]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [ ]:
!ollama pull llama3

In [13]:
import ollama  # Ollama ile Llama 3 kullanımı için
def summarize_with_llama3(text):
    """
    Uses Llama 3 to summarize the given text via Ollama.
    """
    prompt = f"Summarize the following website content:\n\n{text}"
    
    response = ollama.chat(model="llama3", messages=[{"role": "user", "content": prompt}])
    return response['message']['content']

In [14]:
url = "https://www.bbc.com/news/articles/czd31157m31o"  

website = Website(url)
summary = summarize_with_llama3(website.text)

print("\n📰 Website Summary:\n")
print(summary)


📰 Website Summary:

The article reports on a change in tone from US President Trump towards Russian President Putin, who was previously not criticized by Trump. The Russian president had suggested an interim government in Ukraine, which could replace Ukrainian President Volodymyr Zelensky. Trump expressed anger and frustration with Putin's move, saying he is "pissed off" and threatening to impose a 50% tariff on countries buying Russian oil if Putin does not agree to a ceasefire.

Russia's spokesperson, Dmitry Peskov, attempted to downplay the tensions between the two leaders, stating that Russia continues to work with the US side and that there are no plans for a call between Putin and Trump this week. However, Peskov left open the possibility of a call if necessary.

The article also mentions that US and Russian officials have been in talks for several weeks to try to reach a deal to end the war in Ukraine, but Trump's criticism of Putin marks a change in tone towards the Russian pr